
# TensorFlow基本操作
```
Author: PeterSansan
Project: https://github.com/PeterSansan/TF_ACTION
Time : 2017.04.25
```

### [目录]
- [1、算法运算比较GPU与CPU的性能](#1) 
- [2、另一种会话形式](#2)
- [3、assign赋值函数](#3)
- [4、fetch取数据 ](#4)
- [5、feed填数据](#5) 
- [6、get_variable与Variable的一点区别](#6)
- [7、argmax](#7)
- [8、dropout](#8)
- [9、保存数据](#9)
- [10、读取数据](#10)
- [11、变量的赋值不要用run](#11)
- [12、print输出到文件](#12)
- [13、初始化函数](#13)
- [14、命名域与共享变量](#14)
- [15、查看CPU与GPU使用情况](#15)
- [16. tf.reshape](#16)
- [17.tf.transpose](#17)
- [18.tf.gather](#18)
- [19.加减乘除](#19)

In [1]:
# -*- coding:utf-8 -*-
from __future__ import print_function

import tensorflow as tf
import numpy as np
import time
import sys

### <span id="1">1.乘法运算比较GPU与CPU的性能</span>

In [2]:
begin=time.time() # 时间开始
with tf.Session() as sess0:
    with tf.device("/gpu:0"):#g 不加这个话就不能控制用CPU还是GPU，默认是GPU
        matrix1 = np.random.rand(1000,1500).astype(np.float32)
        matrix2 = np.random.rand(1500,1000).astype(np.float32)
        product = tf.matmul(matrix1, matrix2)
        result = sess0.run(product)

end = time.time()  # 时间结束
print("Spend time %f s" %(end - begin))

Spend time 3.600356 s


In [3]:
begin=time.time() # 时间开始
with tf.Session() as sess0:
    with tf.device("/cpu:0"):
        matrix1 = np.random.rand(1000,1500).astype(np.float32)
        matrix2 = np.random.rand(1500,1000).astype(np.float32)
        product = tf.matmul(matrix1, matrix2)
        result = sess0.run(product)

end = time.time()  # 时间结束
print("Spend time %f s" %(end - begin))

Spend time 0.115566 s


<font color="red">上面跑出来的结果可以看出GPU比CPU要快一些</font>

### <span id="2">2.另一种会话形式</span>
前面那种会话是在模型与数据准备好的情况下，开启会话的，一般用`with tf.Session() as sess:`包含在里面，其实还有另外一种交互式更多好的会话形式。

In [4]:
#deploy a session
sess = tf.InteractiveSession()

#design the grape
matrix1 = np.random.rand(200,150).astype(np.float32)
matrix2 = np.random.rand(150,200).astype(np.float32)
product = tf.matmul(matrix1, matrix2)

#run the operation
print(product.eval())

sess.close()

[[ 33.65610123  34.5807457   32.51171112 ...,  31.12764931  38.69446945
   36.99295807]
 [ 35.17318726  35.67451859  33.52633667 ...,  32.91594315  36.71580505
   37.40443802]
 [ 37.40917969  38.15063858  35.33099747 ...,  36.77067184  41.51041412
   38.43318939]
 ..., 
 [ 39.80768585  38.88362503  37.45733261 ...,  35.96920013  42.38987732
   40.98935699]
 [ 35.8903389   36.27746582  33.28411865 ...,  36.44659424  39.87301636
   37.83668518]
 [ 35.47333527  36.97055817  32.76354218 ...,  33.64844894  36.50693893
   37.90606689]]


### <span id="3">3.assign赋值函数</span>
实现了计算器功能

In [5]:
#design the graph
state = tf.Variable(0, name="counter")

one = tf.constant(1)
new_value = tf.add(state, one)
update = tf.assign(state, new_value)  #赋值

#initialization
init_op = tf.global_variables_initializer()

#run 
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(state))
    for _ in range(4):
        sess.run(update)
        print(sess.run(state))

0
1
2
3
4


### <span id="4">4.fetch取数据</span>
这种方法实际上我们上面一直在用

In [6]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)    # 定义三个常量

intermed = tf.add(input2, input3)
mul = tf.multiply(input1, intermed)

with tf.Session() as sess:
    result = sess.run([mul, intermed])
    print(result)  # 取两个结果
    result = sess.run([intermed])
    print(result)   # 取一个结果

[21.0, 7.0]
[7.0]


### <span id="5">5.feed填数据</span>

In [7]:
input1 = tf.placeholder(tf.float32,shape=(3, 3))
input2 = tf.placeholder(tf.float32,shape=(3, 3))
output = tf.matmul(input1, input2)#matmul is different mul

with tf.Session() as sess:
    rand_array = np.ones([3, 3])
    print(sess.run([output], feed_dict={input1: rand_array,input2: rand_array}))

[array([[ 3.,  3.,  3.],
       [ 3.,  3.,  3.],
       [ 3.,  3.,  3.]], dtype=float32)]


### <span id="6">6.get_variable与Variable的一点区别</span>

In [8]:

from pprint import pprint

a = tf.get_variable('a',shape=[5,2])    # 默认 trainable=True
b = tf.get_variable('b',shape=[2,5],trainable=False)
c = tf.constant([1,2,3],dtype=tf.int32,shape=[8],name='c') # 因为是常量，所以trainable=False
d = tf.Variable(tf.random_uniform(shape=[3,3]),name='d')

tvar = tf.trainable_variables()
tvar_name = [x.name for x in tvar]
print(tvar)
# [<tensorflow.python.ops.variables.Variable object at 0x7f9c8db8ca20>, <tensorflow.python.ops.variables.Variable object at 0x7f9c8db8c9b0>]
print(tvar_name)
# ['a:0', 'd:0']

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
pprint(sess.run(tvar))

[<tf.Variable 'counter:0' shape=() dtype=int32_ref>, <tf.Variable 'a:0' shape=(5, 2) dtype=float32_ref>, <tf.Variable 'd:0' shape=(3, 3) dtype=float32_ref>]
[u'counter:0', u'a:0', u'd:0']
[0,
 array([[ 0.91374362,  0.77542281],
       [ 0.09192395, -0.7873472 ],
       [ 0.77425361,  0.6774987 ],
       [ 0.18611455,  0.53725147],
       [ 0.66976511,  0.59764135]], dtype=float32),
 array([[ 0.98069656,  0.57469404,  0.34242105],
       [ 0.81447363,  0.49137688,  0.24920309],
       [ 0.08228135,  0.49962556,  0.02091193]], dtype=float32)]


### <span id="7">7.argmax</span>

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
aa = tf.get_variable(name = 'aa',
                    shape=[3,4],
                    dtype = tf.float32,
                    initializer=tf.random_uniform_initializer(minval=-1,maxval=1))

bb = tf.argmax(input = aa,dimension = 0) # 选出每列中最大值的位置
cc = tf.argmax(input = aa,dimension = 1) # 选出每行中最大值的位置

sess.run(tf.global_variables_initializer())
print(sess.run(aa))
print(sess.run(bb))
print(sess.run(cc))

Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use the `axis` argument instead
[[ 0.60302591  0.72983623 -0.06742501  0.99034882]
 [-0.0453825  -0.3799901   0.69216585  0.25788927]
 [ 0.5958724   0.54000545  0.62644029  0.3175745 ]]
[0 0 1 0]
[3 2 2]


### <span id="8">8、dropout</span>

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
ai = tf.get_variable('ai',shape=[2,5])
bi = ai
ci = ai
a_drop = tf.nn.dropout(ai,0.8)
c_drop = tf.nn.dropout(ci,1)
sess.run(tf.global_variables_initializer())

print(sess.run(ai))
print(sess.run(a_drop))
print(sess.run(c_drop))

[[-0.31626254  0.53986716  0.80050004  0.83001328 -0.18961602]
 [-0.92055213  0.56703258  0.28375387  0.16607571 -0.45236477]]
[[-0.39532816  0.67483395  1.00062501  1.03751659 -0.        ]
 [-1.1506902   0.          0.35469234  0.20759463 -0.56545597]]
[[-0.31626254  0.53986716  0.80050004  0.83001328 -0.18961602]
 [-0.92055213  0.56703258  0.28375387  0.16607571 -0.45236477]]


### <span id="9">9、保存数据</span>

In [11]:
tf.reset_default_graph()
# save to file  
# 下面代码注释掉，因为一个文件保存与读取不要重复了
W = tf.Variable([[1,2,3],[4,5,6]],dtype = tf.float32,name='weight')  
b = tf.Variable([[1,2,3]],dtype = tf.float32,name='biases')  
  
init = tf.global_variables_initializer()  
saver = tf.train.Saver()  
with tf.Session() as sess:  
    sess.run(init)  
    save_path = saver.save(sess,"./save_par/save_par.ckpt")  
    print ("save to path:",save_path)  
    print ("weights:",sess.run(W))  
    print ("biases:",sess.run(b)) 

save to path: ./save_par/save_par.ckpt
weights: [[ 1.  2.  3.]
 [ 4.  5.  6.]]
biases: [[ 1.  2.  3.]]


### <span id="10">10、加载数据</span>

In [12]:
#tf.reset_default_graph()
#W = tf.Variable(np.arange(6).reshape((2,3)),dtype = tf.float32,name='weight')  
#b = tf.Variable(np.arange(3).reshape((1,3)),dtype = tf.float32,name='biases')  
  
#saver = tf.train.Saver()  
#with tf.Session() as sess:  
#        saver.restore(sess,"./save_par/save_par.ckpt")  
#        print ("weights:",sess.run(W))  
#        print ("biases:",sess.run(b))  

### <span id="11">11、变量的赋值不要用run</span>

In [13]:
tf.reset_default_graph()
t1 = time.time()  
x = tf.Variable([1.0])  
b =1.0  
start1 = time.time()  
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())  
    #通过sess.run的方式读变量  
    for step in range(5000):  
        res = sess.run(x)  
    print("通过sess.run的方式读变量所需的时间:",time.time()-start1)  
    start2 = time.time()  
    for step in range(5000):  
        a = b  
    print("通过直接赋值的手段读变量所需的时间:",time.time()-start2)  

通过sess.run的方式读变量所需的时间: 0.924973011017
通过直接赋值的手段读变量所需的时间: 0.000540018081665


### <span id="12">12、print输出到文件</span>
这个功能不是TensorFlow的，放在这里只是它也很常用到

In [14]:
f=open('a.txt','a+') # 追加

old=sys.stdout #将当前系统输出储存到一个临时变量中
sys.stdout=f  #输出重定向到文件
print('Hello weird') #测试一个打印输出
sys.stdout.flush() # 刷新文件流
sys.stdout=old #还原原系统输出
f.close() 
print(open('a.txt','r').read())

# 第二种方法
# f=open('test.txt','a+')
# s= '123'
# abc= '456'
# print >> f, s,abc
# f.close()

Hello weird
Hello weird
Hello weird
Hello weird
Hello weird
Hello weird



### <span id="13">13、使用初始化函数</span>

In [15]:
value = [0,1,2,3,4,5,6,7]
#value = np.array(value)  # 注释掉没有影响
#value = value.reshape([2,4])
init = tf.constant_initializer(value)
print('fitting shape:')
tf.reset_default_graph()
with tf.Session():
  x = tf.get_variable('x',shape=[2,4],initializer=init)  # 这个例子不实用，因为是确定的数值，不是随机数，完全可以用是Variable代替
  x.initializer.run()
  print(x.eval())

fitting shape:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]]


In [16]:
print('larger shape:')  #小型的初始化是不允许的，也就是说shape小于原来的大小
tf.reset_default_graph()
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

larger shape:
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 7.  7.  7.  7.]]


In [17]:
print('random_normal_initializer:')
tf.reset_default_graph()
init = tf.random_normal_initializer(mean=0.0,stddev=1.0) #正态分布函数
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

random_normal_initializer:
[[ 0.32421499 -1.00479341  0.26118875 -2.25137758]
 [-0.84631264 -0.2015401  -0.5502848  -0.18265748]
 [-0.14240623  1.87447703 -0.38946101 -0.75325561]]


In [18]:
print('truncated_normal_initializer:')    # 截断正态分布
tf.reset_default_graph()
init = tf.truncated_normal_initializer(mean=0.0,stddev=1.0)
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

truncated_normal_initializer:
[[ 0.45091966  0.54714847 -0.44627866  1.99480319]
 [ 0.59979331 -0.79542363 -0.9042269   0.40124264]
 [-0.20564921 -0.25250092  0.71303296 -0.73212081]]


In [19]:
print('random_uniform_initializer:')
tf.reset_default_graph()
init = tf.random_uniform_initializer(minval=0,maxval=None) #均匀分布随机数
with tf.Session():
  x = tf.get_variable('x',shape=[3,4],initializer = init)
  x.initializer.run()
  print(x.eval())

random_uniform_initializer:
[[ 0.79297531  0.25500691  0.3216548   0.62649357]
 [ 0.5831902   0.22098529  0.34389579  0.14673936]
 [ 0.49860525  0.50720167  0.89963949  0.09365726]]


```
【说明】：
tf.random_normal | tf.truncated_normal | tf.random_uniform
tf.random_normal(shape,mean=0.0,stddev=1.0,dtype=tf.float32,seed=None,name=None)
tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
tf.random_uniform(shape,minval=0,maxval=None,dtype=tf.float32,seed=None,name=None)
这几个都是用于生成随机数tensor的。尺寸是shape
random_normal: 正太分布随机数，均值mean,标准差stddev
truncated_normal:截断正态分布随机数，均值mean,标准差stddev,不过只保留[mean-2*stddev,mean+2*stddev]范围内的随机数
random_uniform:均匀分布随机数，范围为[minval,maxval]
```

### <span id="14">14、命名域与共享变量</span>
实际上，tf.name_scope与tf.variable_scope都可以用来声明命名域，但一般
```
tf.name_scope与tf.Variable一同使用
tf.variable_scope与tf.variable_scope一同使用
```

In [20]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# 不同的命名域与变量
with tf.variable_scope("foo"):
  with tf.variable_scope("bar"):
    v = tf.get_variable("v", [1])
    print(v.name)

print(v.name)

with tf.variable_scope("foo1"):
  with tf.variable_scope("bar1"):
    v = tf.get_variable("v", [1])
    print(v.name)
print(v.name)


foo/bar/v:0
foo/bar/v:0
foo1/bar1/v:0
foo1/bar1/v:0


In [21]:
# 变量共享（相同变量）
tf.reset_default_graph()
sess = tf.InteractiveSession()
with tf.variable_scope("xxx"):
  a = tf.get_variable("a",[1])
with tf.variable_scope("xxx",reuse=True):   # 采用xxx/a的值
  a1 = tf.get_variable("a",[1])
print(a,a1)
sess.run(tf.global_variables_initializer())
print(a.eval(),a1.eval()) #如果初始化为None,则会采用variable_scope的初始化值，
# 如果也是None,则采用uniform_unit_scaling_initializer
assert a==a1  # a , a1 是一样的

<tf.Variable 'xxx/a:0' shape=(1,) dtype=float32_ref> <tf.Variable 'xxx/a:0' shape=(1,) dtype=float32_ref>
[-0.67850912] [-0.67850912]


In [22]:
# 变量共享（相同变量）另一种写法
tf.reset_default_graph()
with tf.variable_scope("yyy") as scope:
    v = tf.get_variable("v", [1])
    scope.reuse_variables()
    v1 = tf.get_variable("v", [1])
assert v1 == v

In [23]:
# 为防止在没有使用reuse的情况下出现相现的共享变量，则会弹出异常,如下面是有错误的
# with tf.variable_scope("zzz"):
  # v = tf.get_variable("v",[1])
  # v1 = tf.get_variable("v",[1])
  # Raises ValueError("...v already exists ...").
  
# 为防止在使用reuse的情况下引用了之前没有的共享变量，则会弹出异常，如下面是有错误的
# with tf.variable_scope("aaa",reuse=True):
  # v = tf.get_variable("v",[1])
  # Raises ValueError("... v does not exists...").

### <span id="15">15.查看CPU与GPU使用</span>

```
CPU:
    sensors   or   watch sensors
```

```
GPU:
    nvidia-smi --query-gpu=temperature.gpu --format=csv,noheader
    or 
    nvidia-smi -a  # 可以看到显示的所有信息，包括什么温度会关自动关闭
    or
    watch -n 1 nvidia-smi  # 1秒的频率
```

### <span id="16">16.tf.reshape</span>
```
tf.reshape(tensor,shape,name=None)

顾名思义，就是将tensor按照新的shape重新排列。一般来说，shape有三种用法： 
如果 shape=[-1], 表示要将tensor展开成一个list 
如果 shape=[a,b,c,…] 其中每个a,b,c,..均>0，那么就是常规用法 
如果 shape=[a,-1,c,…] 此时b=-1，a,c,..依然>0。这表示tf会根据tensor的原尺寸，自动计算b的值。 
官方给的例子已经很详细了，我就不写示例代码了
```

```
# 官方例子
# tensor 't' is [1, 2, 3, 4, 5, 6, 7, 8, 9]
# tensor 't' has shape [9]
reshape(t, [3, 3]) ==> [[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]

# tensor 't' is [[[1, 1], [2, 2]],
#                [[3, 3], [4, 4]]]
# tensor 't' has shape [2, 2, 2]
reshape(t, [2, 4]) ==> [[1, 1, 2, 2],
                        [3, 3, 4, 4]]

# tensor 't' is [[[1, 1, 1],
#                 [2, 2, 2]],
#                [[3, 3, 3],
#                 [4, 4, 4]],
#                [[5, 5, 5],
#                 [6, 6, 6]]]
# tensor 't' has shape [3, 2, 3]
# pass '[-1]' to flatten 't'
reshape(t, [-1]) ==> [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6]

# -1 can also be used to infer the shape
# -1 is inferred to be 9:
reshape(t, [2, -1]) ==> [[1, 1, 1, 2, 2, 2, 3, 3, 3],
                         [4, 4, 4, 5, 5, 5, 6, 6, 6]]

# -1 is inferred to be 2:
reshape(t, [-1, 9]) ==> [[1, 1, 1, 2, 2, 2, 3, 3, 3],
                         [4, 4, 4, 5, 5, 5, 6, 6, 6]]

# -1 is inferred to be 3:
reshape(t, [ 2, -1, 3]) ==> [[[1, 1, 1],
                              [2, 2, 2],
                              [3, 3, 3]],
                             [[4, 4, 4],
                              [5, 5, 5],
                              [6, 6, 6]]]
```

### <span id="17">17.tf.transpose</span>
转置函数：

转见：[csdn](http://blog.csdn.net/u010417185/article/details/51900441)

### <span id = "18">18.tf.gather</span>
筛选函数：

转见：[csdn](http://blog.csdn.net/guotong1988/article/details/53172882)

### <span id = "19">19.加减乘除</span>

In [24]:
tf.reset_default_graph()
a = tf.constant(5.5)
b = tf.constant(6.5)
sess = tf.InteractiveSession()
print("Subtracting a from b:",sess.run(tf.subtract(b, a)))
print("Adding a and b:",sess.run(tf.add(a, b)))
print("Multiplying a and b:",sess.run(tf.multiply(a, b)))
print("Dividing a and b:",sess.run(tf.divide(b, a)))
print("Floor dividing a and b:",sess.run(tf.floor_div(b, a)))

Subtracting a from b: 1.0
Adding a and b: 12.0
Multiplying a and b: 35.75
Dividing a and b: 1.18182
Floor dividing a and b: 1.0
